In this document, we have code for

* Translate code in the form of vcf written by the 1000 Genome people to the raw sequences
* Compare these sequences with reference sequences to create matrices of allele frequency 
* Run PCA analysis on these matrices


In [1]:
### Manual PCA Analysis
### BIOS 26211 Final Project
### Bruno Petrucci and Christian Porras

import numpy as np
import matplotlib.pyplot as plt
import allel

C:\Users\Petrucci\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Following is the code originally created to simulate some random sequences to test the rest of the code

In [2]:
### First create some Reference Sequence just to test the code
Nucleotides = ['A', 'T', 'C', 'G']
ReferenceSequence1 = np.array([np.random.choice(Nucleotides, 10, p=[0.25, 0.25, 0.25, 0.25])])
ReferenceSequence2 = np.array([np.random.choice(Nucleotides, 10, p=[0.25, 0.25, 0.25, 0.25])])
ReferenceSequence = np.append(ReferenceSequence1, ReferenceSequence2, axis=0)
# Of course it's unrealistic for both of them to be random but it's just for testing - this will be the reference

### Create some random sequence to compare
AnalysisSequence1 = np.array([np.random.choice(Nucleotides, 10, p=[0.25, 0.25, 0.25, 0.25])])
AnalysisSequence2 = np.array([np.random.choice(Nucleotides, 10, p=[0.25, 0.25, 0.25, 0.25])])
AnalysisSequence = np.append(AnalysisSequence1, AnalysisSequence2, axis=0)

Here is the function that transforms sequences into matrices with 0, 1 and 2.
# Need to change some things

In [3]:
### Define the function that transforms the sequence into a PCA-ble matrix
def TransformSequence(Sequence, Reference):
    Transform = np.zeros(Sequence.shape[1])
    for i in range(Sequence.shape[1]):
        if (Sequence[0][i] == Reference[0][i] and Sequence[1][i] == Reference[1][i]) or (Sequence[0][i] == Reference[1][i] and Sequence[1][i] == Reference[0][i]):
            Transform[i] = 0
        elif Sequence[0][i] == Reference[0][i] or Sequence[1][i] == Reference[1][i] or Sequence[0][i] == Reference[1][i] or Sequence[1][i] == Reference[0][i]:
            Transform[i] = 1
        else:
            Transform[i] = 2
        
    return Transform

In [4]:
### Test for the sequences generated above
print(ReferenceSequence)
print(AnalysisSequence)
TransformedSequence = TransformSequence(AnalysisSequence, ReferenceSequence)
print(TransformedSequence)

[['G' 'T' 'T' 'A' 'G' 'C' 'G' 'A' 'G' 'A']
 ['C' 'A' 'A' 'G' 'T' 'G' 'C' 'C' 'A' 'A']]
[['A' 'G' 'A' 'C' 'C' 'T' 'G' 'A' 'A' 'A']
 ['T' 'T' 'G' 'C' 'G' 'C' 'C' 'G' 'A' 'A']]
[2. 1. 1. 2. 1. 1. 0. 1. 1. 0.]


In [5]:
### Generate a bunch of sequences to test the PCA analysis
SequencesGenerated = 10
Sequences = np.array([AnalysisSequence for i in range(SequencesGenerated)])

for i in range(1,SequencesGenerated):
    NewSequence = np.array([np.random.choice(Nucleotides, 10, p=[0.25, 0.25, 0.25, 0.25])])
    Sequences[:][:][i] = NewSequence

TransformedSequences = np.zeros((SequencesGenerated, Sequences.shape[2]))

for i in range(SequencesGenerated):
    TransformedSequences[i,:] = TransformSequence(Sequences[i,:,:], ReferenceSequence)

print(TransformedSequences)

[[2. 1. 1. 2. 1. 1. 0. 1. 1. 0.]
 [2. 1. 2. 2. 1. 1. 1. 1. 2. 2.]
 [1. 2. 1. 1. 2. 2. 2. 2. 1. 2.]
 [2. 2. 1. 1. 2. 1. 1. 1. 2. 0.]
 [2. 1. 1. 1. 2. 1. 2. 1. 1. 0.]
 [1. 1. 1. 2. 1. 1. 2. 1. 2. 2.]
 [1. 1. 2. 1. 2. 2. 2. 2. 2. 2.]
 [2. 2. 1. 1. 2. 1. 2. 1. 2. 2.]
 [2. 1. 2. 2. 2. 2. 2. 2. 1. 2.]
 [1. 1. 1. 2. 2. 1. 1. 1. 2. 2.]]


Now we have the actual 1000 genome data, where we will translate and run PCA on

In [ ]:
### Get the actual 1000 Genome 21st Chromosome data
callset = allel.read_vcf('ALL.chr21.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz')

In [ ]:
### Adjust the 0,1,2 matrix
GT = callset['calldata/GT']
GT[GT == 2] = 1
GT[GT == 3] = 1
ConvertedGT = np.sum(GT, axis=2)
print(ConvertedGT.shape)

Finally, we run the PCA

In [ ]:
### Define PCA function (using code from my Lab 1)
def PCA(M, d, k):
    if (d == 0):
        Row = True
    elif (d == 1):
        Row = False
    else:
        return 0
    CovM = np.cov(M, rowvar = Row)
    eVals, eVecs = np.linalg.eig(CovM)
    CoefDet = eVals/sum(eVals)
    
    return eVecs[:,0:k], CoefDet[0:k]

PC, CoefDet = PCA(ConvertedGT, 0, 2)